In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
unmon_features= pd.read_csv('/content/drive/MyDrive/MLproject/unmon_features.csv')
mon_features= pd.read_csv('/content/drive/MyDrive/MLproject/mon_features.csv')
mon_labels = pd.read_csv('/content/drive/MyDrive/MLproject/mon_labels.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
mon_features['label'] = mon_labels.values
unmon_features['label'] = -1

combined_data = pd.concat([mon_features, unmon_features], axis=0)
combined_data.fillna(0, inplace=True)

X = combined_data.drop('label', axis=1)
y = combined_data['label']

print("Combined Data Shape:", combined_data.shape)
print("Feature Matrix Shape (X):", X.shape)
print(X.head)
print("Target Vector Shape (y):", y.shape)

Combined Data Shape: (29000, 20)
Feature Matrix Shape (X): (29000, 19)
<bound method NDFrame.head of       total_num  total_sum_dir  total_avg  inpkt_num  inpkt_avg  inpkt_sum  \
0         19000        -603648   4.315749       1300   4.292023    5579.63   
1         19000        -183296   4.611815        438   4.673242    2046.88   
2         19000        -574464   5.488969       1240   5.459452    6769.72   
3         19000        -615424   5.995512       1324   5.948693    7876.07   
4         19000        -602112   4.698869       1291   4.663873    6021.06   
...         ...            ...        ...        ...        ...        ...   
9995      10000       -1928704  12.494690       4180  12.360713   51667.78   
9996      10000       -2158592  13.844609       4663  13.729719   64021.68   
9997      10000        -124416   3.568144        302   3.251291     981.89   
9998      10000        -162304   6.892377        413   6.689346    2762.70   
9999      10000       -4785152   7.916714

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix ,precision_score, recall_score
from sklearn.model_selection import GridSearchCV , cross_val_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision", precision_score(y_test, y_pred, average='micro'))
print("Recall", recall_score(y_test, y_pred, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


#하이퍼파라미터 튜닝(GridSearch)
param_grid = {
    'n_estimators': [50, 100],
    'criterion': ['gini']
}
grid_search = GridSearchCV(estimator = RandomForestClassifier(random_state=0),
                           param_grid = param_grid, cv = 3)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_
y_best_pred = best_rf_model.predict(X_test)
best_accuracy = accuracy_score(y_test, y_best_pred)
best_conf_matrix = confusion_matrix(y_test, y_best_pred)

print("Best Parameters:", best_params)
print(f"Best Model Accuracy: {best_accuracy * 100:.2f}%")
print("Best Model Confusion Matrix:\n", best_conf_matrix)


(23200, 19) (5800, 19)
Accuracy: 0.7905172413793103
Precision 0.7905172413793103
Recall 0.6879168788021678

Classification Report:
               precision    recall  f1-score   support

          -1       1.00      1.00      1.00      1977
           0       0.59      0.54      0.57        35
           1       0.69      0.47      0.56        38
           2       0.80      0.89      0.85        37
           3       0.63      0.67      0.65        36
           4       0.59      0.76      0.67        34
           5       0.85      0.81      0.83        36
           6       0.70      0.94      0.80        35
           7       0.57      0.70      0.63        47
           8       0.88      0.77      0.82        39
           9       0.54      0.59      0.57        32
          10       0.57      0.48      0.52        33
          11       0.78      0.64      0.71        45
          12       0.75      0.88      0.81        43
          13       0.53      0.42      0.47        40
   

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'criterion': 'gini', 'n_estimators': 100}
Best Model Accuracy: 79.00%
Best Model Confusion Matrix:
 [[1977    0    0 ...    0    0    0]
 [   0   19    0 ...    0    0    1]
 [   0    0   18 ...    1    0    0]
 ...
 [   0    0    0 ...   22    0    2]
 [   0    1    0 ...    0   40    0]
 [   0    1    0 ...    0    0   23]]
